In [ ]:
import requests
from bs4 import  BeautifulSoup
from google.colab import drive
import time
drive.mount('/content/drive',force_remount=True)



def formatphonenumber(stri):
    if stri[0]=="0":
        stri="3"+stri[1:]
    if len(stri)<10:
        stri="36"+stri    
    out="+"+stri[0:2]+"-("+stri[2:4]+")-"+stri[4:7]+"-"+stri[7:]
    
    return out

def clearphonenumber(stri):
    out=""
    
    for i in stri:
        if i.isnumeric():
            out=out+i
    return(formatphonenumber(out))

def clearengunit(stri):
    out=""
    
    for i in stri:
        if i in ["0","1","2","3","4","5","6","7","8","9"]:
            out=out+i
    return(out)
def ansicode(sti):
    outstr=""
    for i in sti:
        a=i
        if i=="á":
            a="a"
        if i=="Á":
            a="A"
        if i=="é":
            a="e"
        if i=="É":
            a="E"
        if i=="í":
            a="I"
        if i=="Í":
            a="I"
        if i=="ö":
            a="o"
        if i=="Ö":
            a="O"
        if i=="ő":
            a="o"
        if i=="Ő":
            a="O"
        if i=="ó":
            a="o"
        if i=="Ó":
            a="O"
        if i=="ü":
            a="u"
        if i=="Ü":
            a="u"
        if i=="ú":
            a="u"
        if i=="Ú":
            a="u"
        outstr=outstr+a
    return(outstr)

   
        


def queryallinfo(url):
 
    res=requests.get(url)
    #print (res.text)
    soup=BeautifulSoup(res.text,"html.parser")
    
    phonelist=soup.find_all("p", class_="phone lead")
    if len(phonelist)==2:
        phone=phonelist[1].text
    else:
        phone="+36-99-999-9999"
    
    
    #print(phonelist[1].text)
    agentlist=soup.find_all("div", class_="agentName")
    
    if len(agentlist)==1:
        agent=agentlist[0].text
    else:
        agent="NONE"
    
    #print(agentlist[0].text)
    hazterulet=soup.find_all("li", title="Alapterület")
    #print(terulet)
    
    if len(hazterulet)>0:
        hazterulet=clearengunit(hazterulet[0].div.strong.text)
    else:
        hazterulet=0
    telekterulet= soup.find_all("li", title="Telekterület")
    #print(terulet)
    if len(telekterulet)>0:
        telekterulet=clearengunit(telekterulet[0].div.strong.text)
    else:
        telekterulet=0
    out=(agent,clearphonenumber(phone),hazterulet,telekterulet)
    return(out)


def querycity(city):
    print("------------------------------------------------")
    print("+    ",city.capitalize())
    hnev=ansicode(city).lower()

    prefix="https://www.startlak.hu/"
    url=prefix+hnev+"+elado+haz-hazresz+ar--70000000+dragabb"
    res=requests.get(url)
    #print (res.text)
    soup=BeautifulSoup(res.text,"html.parser")
  
    cardcontainer=soup.find_all("div", class_="cardContainer")
    #print(cardcontainer)
    urllist=[]
    pricelist=[]
    for cc in cardcontainer:
        urllist.append(""+cc.div.div.a["href"])
        pricecontainer=cc.find_all("div",class_="figFav")
        pricelist.append(pricecontainer[0]["data-price"])    
    
 

    for i in enumerate(urllist):
        link=prefix+i[1]
        o=(queryallinfo(link))
        name=o[0]
        num=o[1]
        haz=o[2]
        telek=o[3]
        print(f"{city:>15};{i[0]:>4}; {name:>40}; {num:>18}; {haz:>10}; {telek:>10}; {pricelist[i[0]]:>10}; Ft ;  {link:<1000}",file=openfile)
        #
    print("----------------  END  ------------------------")


        
        

hely=open('/content/drive/My Drive/Ingatlan/helyisegek.txt','r')
for i in hely:
    openfile=open('/content/drive/My Drive/Ingatlan/lista_all4.txt','a')
    telepules=i.strip()
    querycity(telepules)
    openfile.close()
    time.sleep(5)

hely.close()










